In [1]:
# Setting up FRED access
import pandas as pd
from fredapi import Fred
api_key = "b40f40a14d67c7903b44700db6b7b6e7"
fred = Fred(api_key=api_key)

In [2]:
# Giving the FRED series IDs for coincident index each of the 50 states
state_series_ids = [
    "ALPHCI", "AKPHCI", "AZPHCI", "ARPHCI", "CAPHCI", "COPHCI", "CTPHCI", "DEPHCI", "FLPHCI", "GAPHCI",
    "HIPHCI", "IDPHCI", "ILPHCI", "INPHCI", "IAPHCI", "KSPHCI", "KYPHCI", "LAPHCI", "MEPHCI", "MDPHCI",
    "MAPHCI", "MIPHCI", "MNPHCI", "MSPHCI", "MOPHCI", "MTPHCI", "NEPHCI", "NVPHCI", "NHPHCI", "NJPHCI",
    "NMPHCI", "NYPHCI", "NCPHCI", "NDPHCI", "OHPHCI", "OKPHCI", "ORPHCI", "PAPHCI", "RIPHCI", "SCPHCI",
    "SDPHCI", "TNPHCI", "TXPHCI", "UTPHCI", "VTPHCI", "VAPHCI", "WAPHCI", "WVPHCI", "WIPHCI", "WYPHCI"
    ]

state_ids = pd.DataFrame([state_series_ids])

In [3]:
# Blank dataframe setup
state_coincident_index = pd.DataFrame()

# Loop to download data for each state
for series_id in state_series_ids:
    state_data = fred.get_series(series_id)
    state_coincident_index = pd.concat([state_coincident_index, state_data], axis = 1)

# Rename columns by state
state_coincident_index.columns = state_series_ids

In [4]:
# Convert dataset to long format to prepare for joining with other data
state_long = state_coincident_index.stack().reset_index()
state_long['level_0'] = pd.to_datetime(state_long['level_0'])
state_long = state_long.rename(columns={0: 'Coincident_Index', 'level_0': 'Date',   'level_1': 'series_id'})

In [5]:
# Reading in shares of economic activity from each state per Fed district
state_mapping_data = pd.read_csv("state_gdp_shares.csv")
state_mapping_data = pd.DataFrame(state_mapping_data)

In [6]:
# Join the mapping data with the coincident index data
merged_data = pd.merge(state_long, state_mapping_data, on='series_id', how='left')

In [7]:
# Calcuate district coincident scores for each month
district_score_by_date = merged_data.groupby(['Date', 'district']).apply(lambda x: (x['Coincident_Index'] * x['share']).sum()).reset_index()

# Name column for the coincident index score
district_score_by_date.rename(columns={0: 'district_coincident_index'}, inplace=True)   

In [8]:
# Pivot the data to wide format
district_score_wide = district_score_by_date.pivot(index='Date', columns='district', values='district_coincident_index')

# Save the data to a csv file
district_score_wide.to_csv('district_coincident_index.csv')